In [15]:
'''
参考代码：https://blog.csdn.net/weixin_42608414/article/details/88391760
'''
import re
#定义删除除字母,数字，汉字以外的所有符号的函数
def remove_punctuation(line):
    line = str(line)
    if line.strip()=='':
        return ''
    rule = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    line = rule.sub('',line)
    return line

#停用词列表
def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]
    return stopwords

#加载停用词
stopwords = stopwordslist("./stopwords.txt")

In [18]:
import pandas as pd
df = pd.read_csv('./data_lite_big.csv')
df=df[['text','label']]
print("数据总量: %d ." % len(df))
df.sample(10)

数据总量: 3664 .


,text,label
426,嗯，看懂了，但是运用泰勒公式原式分子是不是可以直接等于X十X一X一X,0
2561,极限为，看仔细嗷,1
2502,因为你替换后分母没求导,1
3053,这题怎么做这题怎么做，老师,3
3530,为啥代,3
1296,求大佬帮助解答这两个问题,3
1161,课后习题老师这个题第二个等号为什么可以化成这样,2
1941,x的导数是,1
2451,你不要一次放这么多，你分开放出来我倒是可以给你详解,1
2728,好的，我知道了，谢谢,0


In [19]:
import jieba as jb
#删除除字母,数字，汉字以外的所有符号
df['clean_text'] = df['text'].apply(remove_punctuation)

#分词，并过滤停用词
# df['cut_review'] = df['clean_review'].apply(lambda x: " ".join([w for w in list(jb.cut(x)) if w not in stopwords]))
df['cut_text'] = df['clean_text'].apply(lambda x: [w for w in list(jb.cut(x)) if w not in stopwords])
df.head()

,text,label,clean_text,cut_text
0,请问羊犬追逐问题是在哪一节啊多谢各位兄弟,2,请问羊犬追逐问题是在哪一节啊多谢各位兄弟,"[请问, 羊犬, 追逐, 问题, 是, 在, 哪, 一节, 啊, 多谢, 各位, 兄弟]"
1,洛必达法则习题本部分习题第一题，我做了很多遍都是，各位大佬有做出正确答案的吗可以帮萌新一把吗感谢,0,洛必达法则习题本部分习题第一题我做了很多遍都是各位大佬有做出正确答案的吗可以帮萌新一把吗感谢,"[洛必达, 法则, 习题, 本, 部分, 习题, 第一, 题, 我, 做, 了, 很多遍, ..."
2,这个怎么做啊，求,3,这个怎么做啊求,"[这个, 怎么, 做, 啊, 求]"
3,直接带入,1,直接带入,"[直接, 带入]"
4,为什么B不等于即可怎么保证gx在某一段也不为呢为什么B不等于零我们可以保证gx在自变量的某一...,0,为什么B不等于即可怎么保证gx在某一段也不为呢为什么B不等于零我们可以保证gx在自变量的某一...,"[为什么, B, 不, 等于, 即可, 怎么, 保证, gx, 在, 某, 一段, 也, 不..."


In [22]:
from gensim.models.doc2vec import TaggedDocument
#创建标签化文档
train_tagged = df.apply(
    lambda r: TaggedDocument(words=r['cut_text'], tags=[r['label']]), axis=1)
train_tagged.sample(10)

1758                      ([老师, 请问, 有没有, 方便, 的, 解法], [2])
453              ([洛必达, 法则, 求, 极限, 的, 第九, 题, 怎么, 写], [2])
1763                      ([老师, 有没有, 什么, 方便, 的, 方法], [2])
2563                           ([为什么, 不能, 这么, 做, 呢], [2])
3164                  ([老师, A, 不, 依赖, x, 是, 什么, 意思], [2])
1207    ([我, 知道, 了, c, 不, 对, 因为, 所以, 极限, 为, 无穷大, 不, 存在...
2120             ([如何, 判断, 用, 泰勒, 公式, 的, 几阶, 呢, 老师], [2])
1186                                   ([什么, 是, 驻点], [2])
3151    ([老师, 你好, 老师, 请问, 什么, 时候, 用, 泰勒, 公式, 什么, 时候, 用...
2479                             ([哦, 懂, 了, 谢谢, 老师], [0])
dtype: object

In [79]:
import multiprocessing
from gensim.models import Doc2Vec
from tqdm import tqdm

model_dbow = Doc2Vec(dm=1, vector_size=300, negative=10, hs=0, min_count=2, sample = 1e-3, workers=4, window=3)

100%|██████████| 3664/3664 [00:00<00:00, 3611734.40it/s]


In [80]:
from sklearn import utils
for epoch in tqdm(range(30)):
    model_dbow.train(utils.shuffle([x for x in train_tagged.values]), total_examples=len(train_tagged.values), epochs=70)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 30/30 [03:39<00:00,  7.30s/it]


In [81]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words)) for doc in sents])
    return targets, regressors

y_train, X_train = vec_for_learning(model_dbow, train_tagged)

In [83]:
import numpy as np
X_train = list(X_train)
y_train = np.array(y_train)

In [84]:
import numpy as np

total = np.concatenate((X_train, y_train.reshape(len(y_train), 1)), axis=1)
tp = pd.DataFrame(total)
# tp.to_csv("w2v.csv", header=None, index=None)
tp.to_csv("d2v.csv", header=None, index=None)